In [1]:
from context import *

# Stablizer Formalism (`stabilizer`)

### General Idea: State-Map Duality

Every stabilizer state $\rho$ is dual to a Clifford unitary $U$, such that the state can be generated from the zero state $|00\cdots0\rangle$ as
$$\rho = U|00\cdots0\rangle\langle 00\cdots0|U^\dagger$$.
Both $\rho$ and $U$ describes a stabilizer code:
* $\rho$ is a projection operator that specifies the code subspace of the stabilizer code.
* $U$ is the encoding Clifford unitary that encodes the logical + syndrome qubits to the physical qubits in the stabilizer code.

The package `stabilizer` (based on `paulialg`) provides related functions to represent stabilizer states and Clifford maps. There are two classes defined in this package.

* `stabilizer.CliffordMap`. Since the Clifford unitary $U$ maps Pauli operators to Pauli operators, it is sufficient to specify a Clifford unitary by how each single-qubit Pauli operator transforms under the unitary. Such transformation rules are stored in a table called the Clifford map.

* `stabilizer.StabilizerState`. The stabilizer state is specified by a set of stabilizers and the corresponding destabilizers. Using the binary representation of Pauli operators, they can be stored in a table, called the stabilizer tableau. 

Since both classes need to store a table of Pauli operators, they are both realized as subclasses of `paulialg.PauliList`.

## Basic Usage

### Constructors

#### Construct Clifford Maps

`identity_map(N)` constructs an identity Clifford map on $N$ qubits.

In [2]:
qst.identity_map(4)

CliffordMap(
  X0-> +XIII
  Z0-> +ZIII
  X1-> +IXII
  Z1-> +IZII
  X2-> +IIXI
  Z2-> +IIZI
  X3-> +IIIX
  Z3-> +IIIZ)

`random_pauli_map(N)` samples a random Clifford map made of random single-qubit Clifford gates on $N$ qubits, i.e. $U=\prod_i U_i\in\mathrm{Cl}(2)^N$. Each realization specifies a random local Pauli basis.

In [3]:
qst.random_pauli_map(4)

CliffordMap(
  X0-> +XIII
  Z0-> -ZIII
  X1-> +IZII
  Z1-> +IXII
  X2-> -IIYI
  Z2-> -IIXI
  X3-> +IIIX
  Z3-> +IIIY)

`random_clifford_map(N)` samples a globally random Clifford map on $N$ qubits, i.e. $U\in\mathrm{Cl}(2^N)$. Each realization specifies a random global stabilizer basis.

In [4]:
qst.random_clifford_map(4)

CliffordMap(
  X0-> +XXXZ
  Z0-> -XIXY
  X1-> +IZYZ
  Z1-> -IYYZ
  X2-> +IXXI
  Z2-> -IXYX
  X3-> +XIII
  Z3-> +ZXIX)

`clifford_rotation_map(g)` constructs a Clifford map based for a Clifford rotation $\exp(\mathrm{i}\pi/4 g)$ given its generator $g$.

In [5]:
qst.clifford_rotation_map('-XXYZ')

CliffordMap(
  X0-> +XIII
  Z0-> +YXYZ
  X1-> +IXII
  Z1-> +XYYZ
  X2-> +XXZZ
  Z2-> -XXXZ
  X3-> -XXYY
  Z3-> +IIIZ)

#### Construct Stabilizer States

`maximally_mixed_state(N)` constructs a $N$-qubit maximally mixed state (by setting the density matrix to full rank).
$$\rho=2^{-N}\mathbb{1}.$$

In [6]:
qst.maximally_mixed_state(4)

StabilizerState()

`zero_state(N)` constructs a $N$-qubit all-zero state 
$$\rho=|0\cdots0\rangle\langle 0\cdots0|=\prod_{i}\frac{1+Z_i}{2}.$$

In [7]:
qst.zero_state(4)

StabilizerState(
   +ZIII
   +IZII
   +IIZI
   +IIIZ)

`one_state(N)` constructs a $N$-qubit all-one state 
$$\rho=|1\cdots1\rangle\langle 1\cdots1|=\prod_{i}\frac{1-Z_i}{2}.$$

In [8]:
qst.one_state(4)

StabilizerState(
   -ZIII
   -IZII
   -IIZI
   -IIIZ)

`ghz_state(N)` constructs a $N$-qubit GHZ state
$$\rho = |\Psi\rangle\langle\Psi|, \qquad \text{with }|\Psi\rangle=\frac{1}{\sqrt{2}}(|0\cdots0\rangle+|1\cdots1\rangle).$$

In [9]:
qst.ghz_state(4)

StabilizerState(
   +ZZII
   +IZZI
   +IIZZ
   +XXXX)

`random_pauli_map(N)` samples a $N$ qubit random Pauli state.
$$\rho=U|0\cdots0\rangle\langle 0\cdots0|U^\dagger,\qquad\text{with }U\in \mathrm{Cl}(2)^N.$$

In [10]:
qst.random_pauli_state(4)

StabilizerState(
   -ZIII
   -IXII
   +IIZI
   +IIIZ)

`random_clifford_map(N)` samples a $N$ qubit random Clifford (random stabilizer) state.
$$\rho=U|0\cdots0\rangle\langle 0\cdots0|U^\dagger,\qquad\text{with }U\in \mathrm{Cl}(2^N).$$

In [11]:
qst.random_clifford_state(4)

StabilizerState(
   +YZXZ
   +YXYZ
   -XIIY
   -YIIZ)

`stabilizer_state(...)` is a universal constructor of stabilizer state by specifying all stabilizers.

In [12]:
qst.stabilizer_state('XXY','-YYI')

StabilizerState(
   +XXY
   -YYI)

A hack to inspect the full stabilizer tableau is by converting `StabilizerState` to `PauliList` by

In [13]:
qst.stabilizer_state('XXY','-YYI')[:]

 +ZZI
 +XXY
 -YYI
 +ZXZ
 +IIZ
 +ZIZ

User need to ensure that stabilizers commute with each other, otherwise an error will be raised.

In [14]:
qst.stabilizer_state('XXY','-YYI','IZZ')

ValueError: stabilizers must all commute with each other.

#### State-Map Conversion

Stabilizer states and Clifford maps can be mapped to each other.

In [15]:
rho = qst.stabilizer_state('XXY','-YYI')
rho

StabilizerState(
   +XXY
   -YYI)

In [16]:
rho.to_map()

CliffordMap(
  X0-> +ZXZ
  Z0-> +ZZI
  X1-> +IIZ
  Z1-> +XXY
  X2-> +ZIZ
  Z2-> -YYI)

In [17]:
rho.to_map().to_state()

StabilizerState(
   +ZZI
   +XXY
   -YYI)

* `.to_map()` and `.to_state()` will make new copies of Pauli string data in the memory.
* the information about the rank of the density matrix is lost in the Clifford map, so the back conversion will result in a zero rank stabilizer state.

### `CliffordMap` Methods

#### Map Embedding

`.embed(small_map, mask)` provides the method to embed a smaller Clifford map on a subset of qubits to the current Clifford map. This is a in-place operation. The Clifford map object that provide this method will get modified under the embedding.

**Parameters:**
* `small_map` is a `CliffordMap` object supported on a subset of qubits.
* `mask` is a boolean array specifying the subset of qubits.

In [18]:
cmap = qst.identity_map(6)
cmap

CliffordMap(
  X0-> +XIIIII
  Z0-> +ZIIIII
  X1-> +IXIIII
  Z1-> +IZIIII
  X2-> +IIXIII
  Z2-> +IIZIII
  X3-> +IIIXII
  Z3-> +IIIZII
  X4-> +IIIIXI
  Z4-> +IIIIZI
  X5-> +IIIIIX
  Z5-> +IIIIIZ)

In [20]:
cmap.embed(qst.random_clifford_map(3), numpy.array([True,False,False,True,True,False]))

CliffordMap(
  X0-> -IIIZII
  Z0-> -IIIXII
  X1-> +IXIIII
  Z1-> +IZIIII
  X2-> +IIXIII
  Z2-> +IIZIII
  X3-> +ZIIIII
  Z3-> -YIIIYI
  X4-> -ZIIIXI
  Z4-> +ZIIIZI
  X5-> +IIIIIX
  Z5-> +IIIIIZ)

#### Map Composition

`.compose(other)` returns the composition of the current Clifford map with another Clifford map. This will return a new Clifford map without modifying either of the input maps. The Clifford map object which initiates this method will be the preceeding map in the composition. 

**Parameters:**
* `other` - another `CliffordMap`.

Example: composition of a Clifford rotation with its inverse rotation will be an identity map.

In [22]:
qst.clifford_rotation_map('-XXY').compose(qst.clifford_rotation_map('+XXY'))

CliffordMap(
  X0-> +XII
  Z0-> +ZII
  X1-> +IXI
  Z1-> +IZI
  X2-> +IIX
  Z2-> +IIZ)

#### Map Inversion

`.inverse()` returns the inverse of the current Clifford map. This will return a new Clifford map withoutt modifying the original map. The inverse map is such that its composition with the original map must be identity

In [23]:
cmap = qst.clifford_rotation_map('Y')
cmap

CliffordMap(
  X0-> -Z
  Z0-> +X)

In [24]:
cmap.inverse()

CliffordMap(
  X0-> +Z
  Z0-> -X)

Test on random maps.

In [25]:
cmap = qst.random_clifford_map(4)
cmap.inverse().compose(cmap)

CliffordMap(
  X0-> +XIII
  Z0-> +ZIII
  X1-> +IXII
  Z1-> +IZII
  X2-> +IIXI
  Z2-> +IIZI
  X3-> +IIIX
  Z3-> +IIIZ)

In [26]:
cmap.compose(cmap.inverse()) 

CliffordMap(
  X0-> +XIII
  Z0-> +ZIII
  X1-> +IXII
  Z1-> +IZII
  X2-> +IIXI
  Z2-> +IIZI
  X3-> +IIIX
  Z3-> +IIIZ)

Both left and right composition are identity.

### `StabilizerState` Methods

#### Copy

`.copy()` returns a copy of the state, such that the original state will not be touch by modification on the copy state. It is useful to copy the state for measurement (as measurement changes the state).

Example: measurement can change the state. Copy before measurement can prevent the original state from been changed.

In [ ]:
rho0 = qst.random_clifford_state(5)
rho1 = rho0.copy()
rho1.measure(qst.random_pauli_state(5))
print('pre-measurement state:\n{}\npost-measurement state:\n{}'.format(rho0, rho1))

pre-measurement state:
StabilizerState(
   -IZXII
   -YXYYI
   +XZIIX
   +XXZYZ
   -IXYIY)
post-measurement state:
StabilizerState(
   -IXIII
   -ZIIII
   +IIYII
   +IIIXI
   -IXYIY)


#### Measurement

`.measure(obs)` measure the stabilizer state on a set of commuting observables. The state will gets modified in-place.

**Parameters:**
* `obs` - Observables to measure. The following types are supported:
    * `PauliList` - a list of Pauli operators (user must ensure that operators in the list are commuting, otherwise they can not measured simutaneously).
    * `StabilizerState` - stabilizers of a stabilizer state is always commuting, which can be treated as commuting observables for measurement.
    
**Returns:**
* `out` - measuremnt outcome, can only be $0$, $\pm1$ for independent Pauli observables on stabilizer state.
* `log2prob` - the log2 of the probability of realizing this particular outcome.

**Side Effect:**
The state itself will be updated to the post measurement state.

Example: prepare a state and some observables.

In [26]:
rho = qst.ghz_state(3)
obs = qst.random_pauli_state(3).stabilizers
print('state:\n{}\nobservables:\n{}'.format(rho,obs))

state:
StabilizerState(
   +ZZI
   +IZZ
   +XXX)
observables:
 -XII
 +IXI
 -IIZ


Perform the measurement, return the measurement outcomes and log probability.

In [28]:
rho.measure(obs)

(array([1, 0, 1]), -3.0)

The state will get modified.

In [29]:
rho

StabilizerState(
   +XII
   +IXI
   +IIZ)

**Algorithm Outline**:

We scan over every observable $G_k$. For each observable, we continue to scan over all operators in the stabilizer tableau. If the observable $G_k$ anticommute with
1. at least one active stabilizer (the first of them being $S_p$) $\Rightarrow$ $G_k$ is an *error* operator that take the state out of the code subspace $\Rightarrow$ the measurement will collapse the state to one of the two possible measurement outcomes $G_{k}=\pm 1$ with equal probability, and the state will be *updated*.
2. at least one standby stabilizer or destabilizer (the first of them being $S_p$)$\Rightarrow$ $G_k$ is a *logical* operator that will further stabilize the code subspace $\Rightarrow$ the measurement will activate a new pair of stabilizer and destabilizer, and the state will be *extended*.
3. otherwise, $G_k$ is a *trivial* operator in the code subspace $\Rightarrow$ the measurement is classical, and the state is untouched.

|    | `update`? | `extend`? |
|----|-----------|-----------|
| 1. | `True`    | `False`   |
| 2. | `True`    | `True`    |
| 3. | `False`   | `False`   |

* *update*: 
    * $G_k$ must replace $S_p$ to be the active stabilizer. But to mantain its algebraic relatiion with the destabilizer, the original $S_p$ can be promoted to become the corresponding destabilizer. Such that $S_\tilde{p}\leftarrow S_p$, $S_p \leftarrow G_k$ ($\tilde{p}$ denotes the dual row of $p$)
    * The sign of $G_k$ is randomly asigned with half-to-half probability.

* *extend*:
    * The number of logical qubit will be reduced by one $r\leftarrow r-1$.
    * To include new stabilizer-destabilizer pair to the system, apart from the steps in the update algorithm, we also need to bring the new stabilizer $S_p$ to row-$r$ and the new destabilizer $S_{\tilde{p}}$ to row-$(N+r)$. 
    
If update (including extension) did not happen after scanning through all the stabilizers and standby destabilizers, then we know that the measurement is classical ($G_k$ is trivial in code subspace, it must belong to the stabilizer group). So we continue to scan over the remaining active destabilizer. For each active destabilizer $S_j$ that anticommute with $G_k$, it indicates that $G_k$ contains the component of the corresponding active stabilizer $S_{\tilde{j}}$, which should then be collected. Finally the measuremnt outcome $x_k=0,1$ is such that the following equation holds
$$(-)^{h_k+x_k}G_k=\prod_{\tilde{j}}(-)^{b_{\tilde{j}}}S_{\tilde{j}}.$$

#### Expect

`.expect(obs)` evaluates the expectation values of oservables on the stabilizer state. The state will not be modified.

**Parameters:**
* `obs` - Observables in question. The following types are supported:
    * `PauliList` - a list of Pauli operators (no need to be commuting).
    * `StabilizerState` - stabilizers of a stabilizer state is always commuting, which can be treated as commuting observables for measurement.
    

**Returns:**
* `xs` - expectation values in correspondance to `obs`.

Example: use the state and the observales above. The result should be consistent with the measurement outcome ($0\to 1,1\to-1$).

In [30]:
rho.expect(obs)

array([-1,  1, -1])

The expectation values of Pauli operators on stabilizer states are either $0$ or $\pm1$. The expectation value will be $\pm1$ only if the corresponding observable is in the stabilizer group.

#### Entropy

`.entropy(subsys)` calculates the entanglement entropy of the state in a subsystem specified by mask. 

**Parameters:**
* `subsys` - Either a boolean vector specifying the subsystem: `True` = inside, `False` = outside. Or a list of qubit indices in the subsystem.

**Returns:**
* `entropy` - the entanglement entropy $S_{A}(\rho)$ in unit of bit ($\log_2$ base).

Example:

In [3]:
rho = qst.ghz_state(5)
rho.entropy([True,False,True,False,False]), rho.entropy([0,2])

(1, 1)

**Algorithm Outline:**

Definition of entanglement entropy:
$$S_A(\rho)=-\mathrm{Tr}_{A}\rho_A\log_2\rho_A,$$
where $\rho_A = \mathrm{Tr}_{\bar{A}}\rho$. For stabilizer states the entanglement spectrum is flat, so there is no difference between any Renyi orders. The entanglement entropy can be calculated by counting stabilizers.

Let $\Sigma=\{S_k\}$ be the set of stabilizers that stabilize the state $\rho$. Let $\Sigma_A=\mathrm{Tr}_{\bar{A}}\Sigma$ be the subset of stabilizers supported in $A$, $\Sigma_{\bar{A}}=\mathrm{Tr}_{A}\Sigma$ be the subset of stabilizers supported in $\bar{A}$, and $\Sigma_{A,\bar{A}}=\Sigma-\Sigma_{A}-\Sigma_{\bar{A}}$ be the subset of stabilizers across $A$ and $\bar{A}$. Stabilizers in $\Sigma_A$ are definitely stabilizers of subsystem $A$. Howerever, there can be hidden stabilizers of $A$ in $\Sigma_{A,\bar{A}}$. 

To reveal the hidden stabilizers, we can rewrite each stabilizer $S_k\in \Sigma_{A,\bar{A}}$ as a direct product $S_{k}^{A}\otimes S_{k}^{\bar{A}}$ explicitly, where $S_{k}^{A(\bar{A})}$ is the part of the Pauli operator $S_{k}$ that acts on the subsystem $A$($\bar{A}$). While $S_{k}$ are independent and commuting with each other, their restrictions $S_{k}^{A}$ may not be independent and commuting. The statement is that the hidden stabilizers are the generators of the center of the Pauli group generated by $S_{k}^{A}$, i.e. the hidden stabilizers are independnt and commuting elements made of nontrivial products of $S_{k}^{A}$.

To reveal the center, we can first construct the anticommutation indicator matrix
$$M_{k,k'}=\left\{\begin{array}{ll}1 & \{S_{k}^{A},S_{k}^{\bar{A}}\}=0,\\ 0 & \text{otherwise}.\end{array}\right.$$
It can be shown that the center corresponds to the dimension of the modulo-2 null space of the integer matrix $M$.

* If $\rho$ is a mixed state: the entanglement entropy = # of subsystem qubits - # of strictly inside stabilizers - # of hidden stabilizers (nullity of $\Sigma_{A,\bar{A}}|_{A}$)
  $$S_{A}(\rho) = |A|-|\Sigma_{A}|-|\mathrm{null}(\Sigma_{A,\bar{A}}|_{A})|.$$

#### Tokenize

`.tokenize()` tokenize the stabilizer basis. This could be useful for machine learning tasks, where tokens can be processed by NLP techniques.

**Rules:**
* 0 = I
* 1 = X
* 2 = Y
* 3 = Z
* 4 = +
* 5 = -
* 6 = +i
* 7 = -i

Example:

In [9]:
rho = qst.random_clifford_state(5)
print('state:\n{}'.format(rho))
rho.tokenize()

state:
StabilizerState(
   -YIIII
   +IYZXZ
   +IIZYY
   +YZYZY
   +YIIXZ)


array([[2, 0, 0, 0, 0, 5],
       [0, 2, 3, 1, 3, 4],
       [0, 0, 3, 2, 2, 4],
       [2, 3, 2, 3, 2, 4],
       [2, 0, 0, 1, 3, 4]])

#### Sample

`.sample(L)` sample $L$ stabilizers from the stabilizer group.

Example: stabilizers are returned as a `PauliList`.

In [10]:
rho.sample(3)

 -IZYZY
 +IXYIZ
 -IYIYY

The sampled stabilizers can be further tokenized.

In [11]:
rho.sample(3).tokenize()

array([[0, 0, 0, 0, 0, 4],
       [2, 2, 3, 0, 0, 4],
       [2, 2, 3, 1, 3, 5]])

## Algorithm Details

### Internal Representations

#### Stabilizer State

A $[[N,r]]$ stabilizer state ($N$ physical qubits encoding $r$ logical qubits) is describe by the **density matrix** of the following form:
$$\rho = \frac{1}{2^r}\prod_{k=1}^{N-r}\frac{1+(-)^{b_k}S_k}{2}=\frac{1}{2^N}\sum_{S\in\mathcal{S}}S.$$
* Each stabilizer $S_k$ is a (non-trivial) Pauli operator defined on totally $N$ qubits. The stabilizers commute with each other $[S_k,S_{k'}]=0$. They generate an Abelian subgroup $\mathcal{S}=\{\prod_{k=1}^{N-r} S_k^{a_k}|a_k=0,1\}$ of the $N$-qubit Pauli group, called the *stabilizer group*.
* Each sign indicator $b_k=0,1$ is a binary variable specifying the eigen space of the stabilizer.
* There are totally $N-r$ stabilizers for a $[N,r]$ stablizer code (of code rate: $r/N$). The simultaneous eigenspace of all stabilizers constitutes the *code subspace*. 
* The code subspace is $2^r$ dimensional (which is also the rank of the density matrix $\rho$). The stabilizer state $\rho$ is always defined to be the maximally mixed state in the code subspace, such that $\rho$ is also the **projection operator** that projects any state into the code subspace.

#### Stabilizer Tableau

Each stabilizer state is internally stored in the form of a **stabilizer tableau** $S$, together with the **sign indicator** $b$. For a $[[N,r]]$ stabilizer state, its stabilizer tableau is a $2N\times 2N$ matrix of the following structure

<img src="fig_tableau.png" alt="stabilizer tableau." style="width: 320px;"/>

* Each row is a binary representation $(x,z)$ of a Pauli oparator $\sigma_{(x,z)}$.
* Totally $2N$ Pauli operators grouped into $N$ stabilizers and $N$ destabilizers, such that for $i,j=0,\cdots,2N-1$:
$$\sigma_{g_{i}}\sigma_{g_{j}}=(-)^{\delta_{i+N,j}-\delta_{j+N,i}}\sigma_{g_{j}}\sigma_{g_{i}},$$
i.e. the $i$th stabilizer only anticommute with the $i$th destabilizer and they commute with all the other operators in the tableau.
* The rows $r:N$ correspond to the $N-r$ active stabilizers $S_k$, which stabilize the code subspace (impleted as projection operators). The rows $0:r$ corresponds to the $r$ standby (inactive) stabilizers that does not realy stabilizer the code subspace (but they will act as logical operators in the code subspace).
* The rows $N+r:2N$ correspond to the $N-r$ active destabilizers that anticommute with the active stabilizers. The rows $N:N+r$ correspond to the $r$ standby destabilizers taht anticommute with the standby stabilizers.

Although the stabilizer state is only specified by the active stabilizers, the other operators in the stabilizer tableau are still important in order to complete the operator basis. Such that the tableau can specify an unitary operator in the Clifford group that generate the state. The algorithm must mantain the algebraic structure betwen all stabilizers and destabilizers while updating the tableau.

Example: take a stabilizer state

In [14]:
rho = qst.random_clifford_state(5)
rho

StabilizerState(
   -XIXXI
   -ZZYXI
   -ZZYIZ
   -ZIYIZ
   +IIZZX)

The stabilizer tableau is given by `StabilizerState.gs`

In [15]:
rho.gs

array([[1, 0, 0, 0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 0, 1, 1, 1, 0, 0, 0, 1],
       [0, 1, 0, 0, 1, 1, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0, 1, 1, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [1, 1, 0, 1, 1, 0, 1, 1, 0, 0],
       [1, 1, 1, 0, 1, 0, 0, 1, 0, 1],
       [1, 1, 1, 1, 1, 0, 0, 1, 1, 1],
       [0, 1, 0, 0, 1, 1, 0, 0, 0, 0]])

The phase indicator is a $N$ vector, which keeps the phase (in unit of $\pi/2$) of the stabilizers.

In [16]:
rho.ps

array([2, 2, 2, 2, 0, 2, 0, 0, 0, 0])

### Random Stabilizer Algorithm

The algorithm generates a random stabilizer tableau, corresponding to a uniformly sampled element in the global Clifford group. The problem can be solved iteratively. 
* Let $(\mathcal{S}_{N-1}, \mathcal{D}_{N-1})$ be the sets of stabilizers and destabilizers (paired up) of $(N-1)$ qubits.
* The sets can be expanded to $N$ qubits by
$$\mathcal{S}_{N}:\left\{\begin{array}{ll}
S_0=U (Z\otimes I^{\otimes (N-1)}) U^\dagger & \\
S_{i+1}=U (I\otimes S'_{i}) U^\dagger & \text{for }S_i\in \mathcal{S}_{N-1}
\end{array}\right.$$
$$\mathcal{D}_{N}:\left\{\begin{array}{ll}
D_0=U (\left\{\begin{array}{c}X\\Y\end{array}\right\}\otimes I^{\otimes (N-1)}) U^\dagger & \\
D_{i+1}=U (I\otimes D'_{i}) U^\dagger & \text{for }D_i\in \mathcal{D}_{N-1}
\end{array}\right.$$
where $U$ is a random Clifford rotation on $N$ qubits.
* $U$ can be generated by first sample a random pair of stabilizer $S_0$ and destablizer $D_0$, and then find the Clifford rotion to diagonalize them to the first qubit.

#### Random Stabilizer-Destabilizer Pair

Using binary representation of Pauli operators,
* Generate a random non-trivial stabilizer $S_0$ by sampling a binary array of $2N$ components, excluding the all-0 case. (If all-0 array is sampled, reject it and resample, until the vector is not all zero).
* Generate a random destabilizer $D_0$ by 
    * first sampling a binary array of $2N$ components,
    * if $D_0$ anticommute with $S_0$: we are done,
    * if $D_0$ commute with $S_0$: pick the first nontrivial qubit on which $S_0$ acts, modify the corresponding $D_0$ operator on that qubit to flip the commutation relation. The modification is given by the following table.
    
| S\D  | I 00 | X 10 | Y 11 | Z 01 |
|------|------|------|------|------|
| X 10 | Z 01 | Y 11 | X 10 | I 00 |
| Y 11 | X 10 | I 00 | Z 01 | Y 11 |
| Z 01 | Y 11 | Z 01 | I 00 | X 10 |

The rule is:
$$x_i^{(D)} \to x_i^{(D)} + z_i^{(S)}, \quad z_i^{(D)}\to z_i^{(D)} + x_i^{(S)} + z_i^{(S)}$$

#### Find Clifford Rotation

To diagonalize $S_0\to Z_0$ and $D_0\to X_0\text{ or }Y_0$.
* First diagonalize $S_0$ by
    * If $S_0$ commute with $Z_0$,
        * If $S_0=I_0\otimes A$, take the first non-trivial qubit of $A$, permute it cyclically among $X,Y,Z$ to create $B$, then
        $$X_0\otimes B: S_0\to X_0\otimes AB$$
        * If $S_0=Z_0\otimes A$:
        $$X_0\otimes A:S_0\to Y_0\otimes I$$
        * Now $S_0$ has been transformed to anticommute with $Z_0$.
    * If $S_0$ anticommute with $Z_0$:
       $$S_0Z_0: S_0\to Z_0\otimes I$$
* Now $S_0=Z_0\otimes I$, $D_0$ must have been transformed to the form $D_0=\begin{array}{c}X_0\\ Y_0\end{array}\otimes C$.
* Then diagonalize $D_0$ by:
$$Z_0\otimes C: Z_0\otimes I\to Z_0\otimes I, D_0\to \begin{array}{c}X_0\\ Y_0\end{array}\otimes I.$$

Collect the Clifford roations along the way and apply them in reverse order to scramble the stabilizers in $(\mathcal{S}_{N-1}, \mathcal{D}_{N-1})$ to the $N$ qubit system.

### Classes

#### `StabilizerState` Class

`StabilizerState(pauli_list)` represents a stabilizer state whose stabilizer tableau is specified by the given pauli list.

**Parameters:**
* `pauli_list` is a `PauliList` object containing:
    * `pauli_list.gs`: the stabilizer tableau (stabilizers + destabilizers)
    * `pauli_list.ps`: phase indicators (for both stabilizers and destabilizers)

In [19]:
qst.stabilizer_state('XX','-ZZ').__dict__

{'gs': array([[1, 0, 1, 0],
        [0, 1, 0, 1],
        [0, 0, 0, 1],
        [1, 0, 0, 0]]),
 'ps': array([0, 2, 0, 0]),
 'r': 0}

#### `CliffordMap` Class

`CliffordMap(pauli_list)` represents a Clifford map that maps Pauli operators to Pauli operators, which is specified by how each $X$ and $Z$ operator gets mapped to a generic Pauli operator
$$\begin{array}{l}
X_0 \to P_0,\\
Z_0 \to P_1,\\
X_1 \to P_2,\\
Z_1 \to P_3,\\
\cdots
\end{array}$$
where the image of the map $\{P_k\}$ are given by the Pauli list.

**Parameters:**
* `pauli_list` is a `PauliList` object that specifies the image of the Clifford map.